# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,9.90,93,100,5.36,NZ,1696016372
1,1,fale old settlement,-9.3852,-171.2468,28.24,75,9,5.87,TK,1696016372
2,2,port-aux-francais,-49.3500,70.2167,1.07,71,10,19.92,TF,1696016372
3,3,enewetak,11.3474,162.3373,28.81,78,81,6.03,MH,1696016372
4,4,tazovsky,67.4667,78.7000,-2.12,87,7,7.86,RU,1696016373


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_destinations_df = city_data_df.loc[(city_data_df["Max Temp"].astype(float) <= 25)
                                         & (city_data_df["Max Temp"].astype(float) >= 17)
                                         & (city_data_df["Humidity"].astype(float) <= 70)
                                         & (city_data_df["Humidity"].astype(float) >= 30)
                                         & (city_data_df["Cloudiness"].astype(float) >= 10)
                                         & (city_data_df["Cloudiness"].astype(float) <= 50)]

# Drop any rows with null values
ideal_destinations_df.dropna()

# Display sample data
ideal_destinations_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
107,107,mount darwin,-16.7725,31.5838,22.17,50,41,1.42,ZW,1696016385
114,114,kone,-21.0595,164.8658,19.36,69,25,2.68,NC,1696016386
134,134,burao,9.5221,45.5336,24.98,61,42,1.15,SO,1696016388
195,195,la sarre,48.8002,-79.1996,20.82,38,13,3.22,CA,1696016394
270,270,ceerigaabo,10.6162,47.3680,20.17,41,27,2.32,SO,1696016404
272,272,kapuskasing,49.4169,-82.4331,22.05,35,40,5.14,CA,1696016404
322,322,box elder,41.5005,-112.8764,17.79,40,12,1.58,US,1696016411
348,348,alexandroupoli,40.8475,25.8744,21.90,60,20,1.54,GR,1696016414
389,389,glace bay,46.1969,-59.9570,20.50,66,40,4.12,CA,1696016420
415,415,bamna,23.0833,74.7667,24.44,64,20,2.17,IN,1696016423


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_destinations_df[["City", "Country", "Lat", "Lng", "Humidity"]])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
107,mount darwin,ZW,-16.7725,31.5838,50,
114,kone,NC,-21.0595,164.8658,69,
134,burao,SO,9.5221,45.5336,61,
195,la sarre,CA,48.8002,-79.1996,38,
270,ceerigaabo,SO,10.6162,47.3680,41,
272,kapuskasing,CA,49.4169,-82.4331,35,
322,box elder,US,41.5005,-112.8764,40,
348,alexandroupoli,GR,40.8475,25.8744,60,
389,glace bay,CA,46.1969,-59.9570,66,
415,bamna,IN,23.0833,74.7667,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mount darwin - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
burao - nearest hotel: Liiban hotel burco ( ENG:Shibiin )
la sarre - nearest hotel: No hotel found
ceerigaabo - nearest hotel: Sanaag Hotel
kapuskasing - nearest hotel: Travelodge
box elder - nearest hotel: No hotel found
alexandroupoli - nearest hotel: Athina
glace bay - nearest hotel: No hotel found
bamna - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
sapri - nearest hotel: "Pisacane" Hotel
misratah - nearest hotel: الفندق السياحى
mek'ele - nearest hotel: Union Hotel
isangel - nearest hotel: Tanna Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
107,mount darwin,ZW,-16.7725,31.5838,50,No hotel found
114,kone,NC,-21.0595,164.8658,69,Pacifik Appartels
134,burao,SO,9.5221,45.5336,61,Liiban hotel burco ( ENG:Shibiin )
195,la sarre,CA,48.8002,-79.1996,38,No hotel found
270,ceerigaabo,SO,10.6162,47.3680,41,Sanaag Hotel
272,kapuskasing,CA,49.4169,-82.4331,35,Travelodge
322,box elder,US,41.5005,-112.8764,40,No hotel found
348,alexandroupoli,GR,40.8475,25.8744,60,Athina
389,glace bay,CA,46.1969,-59.9570,66,No hotel found
415,bamna,IN,23.0833,74.7667,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols=['Lng','Lat', 'City', 'Humidity', 'Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)